<a href="https://colab.research.google.com/github/aninsung/2024-project/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [37]:
import pandas as pd
import glob

# 1. 모든 피험자 파일 리스트 가져오기
files = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET/mHealth_subject*.log"))

# 2. 컬럼 이름 정의 (센서 feature 23개 + label)
columns = [
    "chest_acc_x", "chest_acc_y", "chest_acc_z",
    "ecg_1", "ecg_2",
    "chest_gyro_x", "chest_gyro_y", "chest_gyro_z",
    "chest_mag_x", "chest_mag_y", "chest_mag_z",
    "ankle_acc_x", "ankle_acc_y", "ankle_acc_z",
    "ankle_gyro_x", "ankle_gyro_y", "ankle_gyro_z",
    "ankle_mag_x", "ankle_mag_y", "ankle_mag_z",
    "wrist_acc_x", "wrist_acc_y", "wrist_acc_z",
    "label"
]

# 3. 각 파일 읽어서 subject_id 추가
dfs = []
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file, sep="\s+", header=None)  # 공백 구분자로 읽기
    df.columns = columns
    df["subject_id"] = i  # 피험자 번호 추가
    dfs.append(df)

# 4. 모든 피험자 데이터 통합
data = pd.concat(dfs, ignore_index=True)

# 5. 확인
print(data.head())
print(data.shape)



<>:23: SyntaxWarning: invalid escape sequence '\s'
<>:23: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4026660253.py:23: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep="\s+", header=None)  # 공백 구분자로 읽기


   chest_acc_x  chest_acc_y  chest_acc_z     ecg_1     ecg_2  chest_gyro_x  \
0      -9.8184     0.009971      0.29563  0.004186  0.004186        2.1849   
1      -9.8489     0.524040      0.37348  0.004186  0.016745        2.3876   
2      -9.6602     0.181850      0.43742  0.016745  0.037677        2.4086   
3      -9.6507     0.214220      0.24033  0.079540  0.117220        2.1814   
4      -9.7030     0.303890      0.31156  0.221870  0.205130        2.4173   

   chest_gyro_y  chest_gyro_z  chest_mag_x  chest_mag_y  ...  ankle_gyro_y  \
0       -9.6967       0.63077     0.103900     -0.84053  ...       -4.5781   
1       -9.5080       0.68389     0.085343     -0.83865  ...       -4.3198   
2       -9.5674       0.68113     0.085343     -0.83865  ...       -4.2772   
3       -9.4301       0.55031     0.085343     -0.83865  ...       -4.3163   
4       -9.3889       0.71098     0.085343     -0.83865  ...       -4.1459   

   ankle_gyro_z  ankle_mag_x  ankle_mag_y  ankle_mag_z  wrist_

In [38]:
data

,chest_acc_x,chest_acc_y,chest_acc_z,ecg_1,ecg_2,chest_gyro_x,chest_gyro_y,chest_gyro_z,chest_mag_x,chest_mag_y,...,ankle_gyro_y,ankle_gyro_z,ankle_mag_x,ankle_mag_y,ankle_mag_z,wrist_acc_x,wrist_acc_y,wrist_acc_z,label,subject_id
0,-9.8184,0.009971,0.295630,0.004186,0.004186,2.18490,-9.6967,0.63077,0.103900,-0.84053,...,-4.5781,0.187760,-0.44902,-1.01030,0.034483,-2.35000,-1.610200,-0.030899,0,1
1,-9.8489,0.524040,0.373480,0.004186,0.016745,2.38760,-9.5080,0.68389,0.085343,-0.83865,...,-4.3198,0.023595,-0.44902,-1.01030,0.034483,-2.16320,-0.882540,0.326570,0,1
2,-9.6602,0.181850,0.437420,0.016745,0.037677,2.40860,-9.5674,0.68113,0.085343,-0.83865,...,-4.2772,0.275720,-0.44902,-1.01030,0.034483,-1.61750,-0.165620,-0.030693,0,1
3,-9.6507,0.214220,0.240330,0.079540,0.117220,2.18140,-9.4301,0.55031,0.085343,-0.83865,...,-4.3163,0.367520,-0.45686,-1.00820,0.025862,-1.07710,0.006945,-0.382620,0,1
4,-9.7030,0.303890,0.311560,0.221870,0.205130,2.41730,-9.3889,0.71098,0.085343,-0.83865,...,-4.1459,0.407290,-0.45686,-1.00820,0.025862,-0.53684,0.175900,-1.095500,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215740,-12.2440,-2.970600,-0.035772,0.812140,1.096800,0.57061,-2.5602,0.41936,-0.055659,0.64165,...,-16.9660,2.370400,0.10980,-0.99384,0.191810,-6.12870,15.495000,18.856000,0,10
1215741,-10.9220,-3.419000,-0.339280,1.469400,1.285200,5.26840,-4.9331,9.66020,-0.055659,0.64165,...,-13.3900,2.997200,0.10980,-0.99384,0.191810,-10.24200,17.139000,33.920000,0,10
1215742,-9.4842,-3.064300,-1.033700,0.238620,0.891680,0.53805,-5.9706,2.93600,-0.055659,0.64165,...,-11.3790,3.149800,0.10980,-0.99384,0.191810,-12.94400,16.170000,43.262000,0,10
1215743,-8.7889,-2.475700,-0.612290,-0.205130,0.460490,1.56950,-7.9809,-2.05000,-0.085343,0.43715,...,-10.0000,3.000000,0.11961,-0.97331,0.153020,-14.52400,1.849400,43.373000,0,10


In [39]:
import numpy as np
from sklearn.model_selection import train_test_split

# 예: 통합 DataFrame
# data.columns = ['chest_acc_x', ..., 'label', 'subject_id']

subjects = np.arange(1, 11)  # 1~10
folds = [(subjects[i], subjects[i+1]) for i in range(0, 10, 2)]  # 2명씩 그룹

for fold_idx, test_subjects in enumerate(folds, start=1):
    # 테스트셋
    test_data = data[data["subject_id"].isin(test_subjects)]
    # 학습셋
    train_data = data[~data["subject_id"].isin(test_subjects)]

    X_train = train_data.drop(["label", "subject_id"], axis=1).values
    y_train = train_data["label"].values
    X_test = test_data.drop(["label", "subject_id"], axis=1).values
    y_test = test_data["label"].values

    print(f"Fold {fold_idx}: Train={X_train.shape}, Test={X_test.shape}")


Fold 1: Train=(956161, 23), Test=(259584, 23)
Fold 2: Train=(963072, 23), Test=(252673, 23)
Fold 3: Train=(979201, 23), Test=(236544, 23)
Fold 4: Train=(1012993, 23), Test=(202752, 23)
Fold 5: Train=(951553, 23), Test=(264192, 23)


In [40]:
# Scale train/ test predictors based on training data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
from collections.abc import Sequence
import numpy as np

Sequence=100
n_steps=50
# 슬라이딩 윈도우 함수
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        X.append(sequences[i:end_ix, :-1])
        y.append(sequences[end_ix-1, -1])
    return np.array(X), np.array(y)

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM

n_classes = data["label"].nunique()
n_features = data.drop(["label","subject_id"],axis=1).shape[1]
timesteps = 100  # frame_size

# CNN 모델
def build_cnn():
    model = Sequential([
        Conv1D(64, 3, activation="relu", input_shape=(timesteps, n_features)),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# LSTM 모델
def build_lstm():
    model = Sequential([
        LSTM(64, input_shape=(timesteps, n_features)),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# CNN-LSTM 모델
def build_cnn_lstm():
    model = Sequential([
        Conv1D(64,3, activation="relu", input_shape=(timesteps, n_features)),
        MaxPooling1D(2),
        LSTM(64),
        Dense(64, activation="relu"),
        Dense(n_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


In [43]:
subjects = np.arange(1,11)
folds = [(subjects[i], subjects[i+1]) for i in range(0,10,2)]

for fold_idx, test_subjects in enumerate(folds, start=1):
    # Train/Test 분리
    test_data = data[data["subject_id"].isin(test_subjects)]
    train_data = data[~data["subject_id"].isin(test_subjects)]

    # 윈도우 생성
    X_train, y_train = create_windows(train_data, frame_size=timesteps, hop_size=50)
    X_test, y_test = create_windows(test_data, frame_size=timesteps, hop_size=50)

    print(f"\n=== Fold {fold_idx}: Test subjects {test_subjects} ===")
    print("Samples:", X_train.shape[0], "(train)", X_test.shape[0], "(test)")

    # 모델 선택
    model = build_cnn()  # CNN, LSTM, CNN-LSTM 중 선택
    # model = build_lstm()
    # model = build_cnn_lstm()

    # 학습
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

    # 평가
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {acc:.4f}")



=== Fold 1: Test subjects (np.int64(1), np.int64(2)) ===
Samples: 19122 (train) 5190 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6683 - loss: 2.8727
Epoch 2/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7490 - loss: 0.6376
Epoch 3/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8001 - loss: 0.4333
Epoch 4/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8279 - loss: 0.3776
Epoch 5/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8421 - loss: 0.3382
Test Accuracy: 0.7414

=== Fold 2: Test subjects (np.int64(3), np.int64(4)) ===
Samples: 19260 (train) 5052 (test)
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6641 - loss: 3.4507
Epoch 2/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7694 - loss: 0.6151
Epoch 3/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8043 - loss: 0.4457
Epoch 4/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8258 - loss: 0.3716
Epoch 5/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8393 - loss: 0.3482
Test Accuracy: 0.7607

=== Fold 3: Test subjects (np.int64(5), np.int64(6)) ===
Samples: 19583 (train) 4729 (test)


...

Epoch 1/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6579 - loss: 3.7124
Epoch 2/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7578 - loss: 0.6449
Epoch 3/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7976 - loss: 0.4672
Epoch 4/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8306 - loss: 0.3738
Epoch 5/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8408 - loss: 0.3395
Test Accuracy: 0.7684

=== Fold 4: Test subjects (np.int64(7), np.int64(8)) ===
Samples: 20258 (train) 4054 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6501 - loss: 3.4542
Epoch 2/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7467 - loss: 0.6820
Epoch 3/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7896 - loss: 0.4897
Epoch 4/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8187 - loss: 0.3905
Epoch 5/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8369 - loss: 0.3590
Test Accuracy: 0.7255

=== Fold 5: Test subjects (np.int64(9), np.int64(10)) ===
Samples: 19030 (train) 5282 (test)
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6193 - loss: 3.3236
Epoch 2/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7331 - loss: 0.6799
Epoch 3/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7798 - loss: 0.4800
Epoch 4/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7913 - loss: 0.4283
Epoch 5/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8131 - loss: 0.3782
Test Accuracy: 0.7709
